# Performance dataframes

In [152]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

## General parameters

In [153]:
instance_key = 'instance'

avg_constr_obj_key = 'constr obj'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'
alns_gap_key = 'gap'

max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'

max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_best_sol_iter_key = 'best found iter'
avg_iter_key = 'iter'

dr_improv_key = 'dr improv (#)'
ls_improv_key = 'ls improv (#)'
best_ls_improv_percent_key = 'best ls improv (%)'
set_part_key = 'sp improv (#)'

best_sol_found_by_key = 'best sol found by'
dr_found_best_sol_key = 'drfb'
ls_found_best_sol_key = 'lsfb'
sp_found_best_sol_key = 'spfb'
cr_found_best_sol_key = 'crfb'

gap_key = 'gap'
preprocess_key = 'preproc'
model_key = 'time'
variables_key = 'variables'

one_exchange_key = 'one exchange'
one_relocate_key = 'one relocate'
two_exchange_key = 'two exchange'
two_relocate_key = 'two relocate'
post_sched_key = 'postpone scheduled'
sched_post_key = 'schedule postponed'
voyage_exchange_key = 'voyage exchange'

project_path = os.path.dirname(os.path.abspath('.'))
directory_path_alns = '/output/solstorm/alns/performance/'
directory_path_exact = '/output/solstorm/arcflow/performance/'

generate_df = True
run_number = 'fifth'

## Functions

In [154]:
def map_instance_to_data_alns(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            avg_constr_obj = history_json['construction_heuristic_objective']
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            best_it = history_json['best_sol_found_in_iteration']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']
            ls_improv = history_json['number_of_improvements_by_local_search']
            ls_improv_percent = history_json['best_improvement_local_search']
            dr_improv = history_json['number_of_improvements_by_destroy_repair']
            best_sol_found_by = history_json['best_solution_found_by']
            dr = 1 if best_sol_found_by == 'destroy_repair' else 0
            ls = 1 if best_sol_found_by == 'local_search' else 0
            sp = 1 if best_sol_found_by == 'set_partitioning' else 0
            cr = 1 if best_sol_found_by == 'construction_heuristic' else 0

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[4]:
                    data[4] = time
                if time < data[5]:
                    data[5] = time
                if it > data[7]:
                    data[7] = it
                if it < data[8]:
                    data[8] = it
                
                data[2] += obj
                data[3] += avg_constr_obj
                data[6] += time
                data[9] += it
                data[10] += best_it
                data[11] += dr_improv
                data[12] += ls_improv
                data[13] += ls_improv_percent
                data[14] += set_part_improv
                data[15] += dr
                data[16] += ls
                data[17] += sp
                data[18] += cr
                data[19] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, obj, avg_constr_obj,
                                                   time, time, time, 
                                                   it, it, it, best_it,
                                                   dr_improv, ls_improv, ls_improv_percent, set_part_improv,
                                                   dr, ls, sp, cr,
                                                   1]
    return instance_to_data

def generate_run_df_alns(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_alns(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_obj_key, avg_constr_obj_key,
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key, avg_best_sol_iter_key,
                               dr_improv_key, ls_improv_key, best_ls_improv_percent_key, set_part_key,
                               dr_found_best_sol_key, ls_found_best_sol_key, sp_found_best_sol_key, cr_found_best_sol_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[19]
        if nbr_sims != 5:
            print(f'{instance} DEVIATES IN SIMULATIONS!')
        
        best_objective = data[0]
        worst_objective = data[1]
        avg_objective = data[2] / nbr_sims
        alns_gap = (avg_objective - best_objective) / avg_objective
        avg_constr_objective = data[3] / nbr_sims
        max_time = data[4]
        min_time = data[5]
        avg_time = data[6] / nbr_sims
        max_iter = data[7]
        min_iter = data[8]
        avg_iter = data[9] / nbr_sims
        avg_best_sol_iter = data[10] / nbr_sims
        avg_dr_improv = data[11] / nbr_sims
        avg_ls_improv = data[12] / nbr_sims
        avg_ls_improv_percent = data[13] / nbr_sims
        avg_set_part_improv = data[14] / nbr_sims
        dr_found_best_sol = data[15]
        ls_found_best_sol = data[16]
        sp_found_best_sol = data[17]
        cr_found_best_sol = data[18]
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_obj_key: avg_objective,
                         alns_gap_key: alns_gap,
                         avg_constr_obj_key: avg_constr_objective,
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_iter_key: avg_iter,
                         avg_best_sol_iter_key: avg_best_sol_iter,
                         dr_improv_key: avg_dr_improv,
                         ls_improv_key: avg_ls_improv,
                         best_ls_improv_percent_key: avg_ls_improv_percent,
                         set_part_key: avg_set_part_improv,
                         dr_found_best_sol_key: dr_found_best_sol,
                         ls_found_best_sol_key: ls_found_best_sol,
                         sp_found_best_sol_key: sp_found_best_sol,
                         cr_found_best_sol_key: cr_found_best_sol})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          alns_gap_key: df[alns_gap_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          dr_found_best_sol_key: df[dr_found_best_sol_key].mean(),
                          ls_found_best_sol_key: df[ls_found_best_sol_key].mean(),
                          sp_found_best_sol_key: df[sp_found_best_sol_key].mean(),
                          cr_found_best_sol_key: df[cr_found_best_sol_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(3)
    return df

def aggregate_df_by_instance_group_alns(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_obj = row[avg_obj_key]
        alns_gap = row[alns_gap_key]
        avg_constr_obj = row[avg_constr_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_iter = row[avg_iter_key]
        avg_best_sol_iter = row[avg_best_sol_iter_key]
        avg_dr_improv = row[dr_improv_key]
        avg_ls_improv = row[ls_improv_key]
        avg_ls_improv_percent = row[best_ls_improv_percent_key]
        avg_set_part_improv = row[set_part_key]
        avg_dr_found_best_sol = row[dr_found_best_sol_key]
        avg_ls_found_best_sol = row[ls_found_best_sol_key]
        avg_sp_found_best_sol = row[sp_found_best_sol_key]
        avg_cr_found_best_sol = row[cr_found_best_sol_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_obj
            data[3] += alns_gap
            data[4] += avg_constr_obj
            data[5] += max_time
            data[6] += min_time
            data[7] += avg_time
            data[8] += max_iter
            data[9] += min_iter
            data[10] += avg_iter
            data[11] += avg_best_sol_iter
            data[12] += avg_dr_improv
            data[13] += avg_ls_improv
            data[14] += avg_ls_improv_percent
            data[15] += avg_set_part_improv
            data[16] += avg_dr_found_best_sol
            data[17] += avg_ls_found_best_sol
            data[18] += avg_sp_found_best_sol
            data[19] += avg_cr_found_best_sol
            data[20] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_obj, alns_gap, avg_constr_obj,
                    max_time, min_time, avg_time, 
                    max_iter, min_iter, avg_iter, avg_best_sol_iter,
                    avg_dr_improv, avg_ls_improv, avg_ls_improv_percent, avg_set_part_improv,
                    avg_dr_found_best_sol, avg_ls_found_best_sol, avg_sp_found_best_sol, avg_cr_found_best_sol,
                    1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', 
                               best_obj_key, worst_obj_key, avg_obj_key, alns_gap_key, avg_constr_obj_key,
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key, avg_best_sol_iter_key,
                               dr_improv_key, ls_improv_key, best_ls_improv_percent_key, set_part_key,
                               dr_found_best_sol_key, ls_found_best_sol_key, sp_found_best_sol_key, cr_found_best_sol_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[20]
        if nbr_sims != 5:
            print(f'{instance_size} INSTANCE_SIZE DEVIATES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_obj_key: data[2] / nbr_sims,
                         alns_gap_key: data[3] / nbr_sims,
                         avg_constr_obj_key: data[4] / nbr_sims,
                         max_time_key: data[5] / nbr_sims,
                         min_time_key: data[6] / nbr_sims,
                         avg_time_key: data[7] / nbr_sims, 
                         max_iter_key: data[8] / nbr_sims,
                         min_iter_key: data[9] / nbr_sims,
                         avg_iter_key: data[10] / nbr_sims,
                         avg_best_sol_iter_key: data[11] / nbr_sims,
                         dr_improv_key: data[12] / nbr_sims,
                         ls_improv_key: data[13] / nbr_sims,
                         best_ls_improv_percent_key: data[14] / nbr_sims,
                         set_part_key: data[15] / nbr_sims,
                         dr_found_best_sol_key: data[16] / nbr_sims,
                         ls_found_best_sol_key: data[17] / nbr_sims,
                         sp_found_best_sol_key: data[18] / nbr_sims,
                         cr_found_best_sol_key: data[19] / nbr_sims,})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          alns_gap_key: df[alns_gap_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          dr_found_best_sol_key: df[dr_found_best_sol_key].mean(),
                          ls_found_best_sol_key: df[ls_found_best_sol_key].mean(),
                          sp_found_best_sol_key: df[sp_found_best_sol_key].mean(),
                          cr_found_best_sol_key: df[cr_found_best_sol_key].mean(),})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(3)
    return df

def map_instance_to_data_exact(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        with open(run_path + file_name) as file:
            exact_json = json.load(file)
        
        obj = exact_json['objective']['incumbent']
        gap = exact_json['objective']['optimality_gap']
        preprocess_runtime = exact_json['runtime']['preprocess_runtime']
        model_runtime = exact_json['runtime']['model_runtime']
        variables = exact_json['variables']['number_of_variables']
        
        if instance_name in instance_to_data:
            print('Multiple versions of same instance!')
        
        instance_to_data[instance_name] = [obj, gap, preprocess_runtime, model_runtime, variables]
    
    return instance_to_data

def generate_run_df_exact(run_name):
    run_path = project_path + directory_path_exact + run_name
    instance_to_data = map_instance_to_data_exact(run_path)

    df = pd.DataFrame(columns=[instance_key, avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    for instance in instance_to_data:
        data = instance_to_data[instance]
        obj = data[0]
        gap = data[1]
        preprocess_runtime = data[2]
        model_runtime = data[3]
        variables = data[4]
        row = pd.Series({instance_key: instance,
                         avg_obj_key: obj,
                         gap_key: gap,
                         preprocess_key: preprocess_runtime,
                         model_key: model_runtime,
                         variables_key: variables})
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(),
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(3)
    return df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(5, len(df_total.columns), 5)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group_exact(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        obj = row[avg_obj_key]
        gap = row[gap_key]
        preprocess_runtime = row[preprocess_key]
        model_runtime = row[model_key]
        variables = row[variables_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += obj
            data[1] += gap
            data[2] += preprocess_runtime
            data[3] += model_runtime
            data[4] += variables
            data[5] += 1 # Number of times encountered instance size
        else:
            instance_size_to_data[instance_size] = [obj, gap, preprocess_runtime, model_runtime, variables, 1]

    df = pd.DataFrame(columns=['instance_group', avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[5]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         avg_obj_key: data[0] / nbr_sims,
                         gap_key: data[1] / nbr_sims,
                         preprocess_key: data[2] / nbr_sims, 
                         model_key: data[3] / nbr_sims,
                         variables_key: data[4] / nbr_sims})
    
        df = df.append(row, ignore_index=True)

        
    mean_row = pd.Series({'instance_group': 'Mean values', 
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(), 
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(3)
    return df

def map_instance_to_data_lso(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            nbr_improv_one_exchange = history_json['number_of_improvements_by_local_search_operators']['one_exchange']
            nbr_improv_one_relocate = history_json['number_of_improvements_by_local_search_operators']['one_relocate']
            nbr_improv_two_exchange = history_json['number_of_improvements_by_local_search_operators']['two_exchange']
            nbr_improv_two_relocate = history_json['number_of_improvements_by_local_search_operators']['two_relocate']
            nbr_improv_post_sched = history_json['number_of_improvements_by_local_search_operators']['postpone_scheduled']
            nbr_improv_sched_post = history_json['number_of_improvements_by_local_search_operators']['schedule_postponed']
            nbr_improv_voy_exchange = history_json['number_of_improvements_by_local_search_operators']['voyage_exchange']
            
            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]
                data[0] += nbr_improv_one_exchange
                data[1] += nbr_improv_one_relocate
                data[2] += nbr_improv_two_exchange
                data[3] += nbr_improv_two_relocate
                data[4] += nbr_improv_post_sched
                data[5] += nbr_improv_sched_post
                data[6] += nbr_improv_voy_exchange
                data[7] += 1
            else:
                instance_to_data[instance_name] = [nbr_improv_one_exchange,
                                                   nbr_improv_one_relocate,
                                                   nbr_improv_two_exchange,
                                                   nbr_improv_two_relocate,
                                                   nbr_improv_post_sched,
                                                   nbr_improv_sched_post,
                                                   nbr_improv_voy_exchange,
                                                   1]
    return instance_to_data

def generate_lso_df(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_lso(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               one_exchange_key, one_relocate_key, 
                               two_exchange_key, two_relocate_key,
                               post_sched_key, sched_post_key,
                               voyage_exchange_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[7]
        
        nbr_improv_one_exchange = data[0] / nbr_sims
        nbr_improv_one_relocate = data[1] / nbr_sims
        nbr_improv_two_exchange = data[2] / nbr_sims
        nbr_improv_two_relocate = data[3] / nbr_sims
        nbr_improv_post_sched = data[4] / nbr_sims
        nbr_improv_sched_post = data[5] / nbr_sims
        nbr_improv_voy_exchange = data[6] / nbr_sims
        
        row = pd.Series({instance_key: instance,
                         one_exchange_key: nbr_improv_one_exchange,
                         one_relocate_key: nbr_improv_one_relocate,
                         two_exchange_key: nbr_improv_two_exchange,
                         two_relocate_key: nbr_improv_two_relocate,
                         post_sched_key: nbr_improv_post_sched,
                         sched_post_key: nbr_improv_sched_post,
                         voyage_exchange_key: nbr_improv_voy_exchange})

        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          one_exchange_key: df[one_exchange_key].mean(),
                          one_relocate_key: df[one_relocate_key].mean(),
                          two_exchange_key: df[two_exchange_key].mean(),
                          two_relocate_key: df[two_relocate_key].mean(),
                          post_sched_key: df[post_sched_key].mean(),
                          sched_post_key: df[sched_post_key].mean(),
                          voyage_exchange_key: df[voyage_exchange_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(3)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key, avg_best_sol_iter_key], 
                axis=1, inplace=True)
        if best_sol_found_by_key in df:
            df.drop([best_sol_found_by_key], axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(12, len(df_total.columns), 12)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def merge_dfs_exact_alns(df_3600, df_600, df_alns):
    df_3600_copy = df_3600.copy()
    df_600_copy = df_600.copy()
    exact_dfs = [df_3600_copy, df_600_copy]
    for exact_df in exact_dfs:
        exact_df.drop([preprocess_key, variables_key], axis=1, inplace=True)

    df_alns_copy = df_alns.copy()
    df_alns_copy.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                       max_time_key, min_time_key, 
                       max_iter_key, min_iter_key, avg_iter_key, avg_best_sol_iter_key, 
                       dr_improv_key, ls_improv_key, best_ls_improv_percent_key, set_part_key, 
                       dr_found_best_sol_key, ls_found_best_sol_key, sp_found_best_sol_key, cr_found_best_sol_key], 
                      axis=1, inplace=True)
        
    df_total = pd.concat([df_3600_copy, df_600_copy, df_alns_copy], axis=1)
    
    li = [i for i in range(4, len(df_total.columns), 4)]
    df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    exact_gaps = []
    for idx, row in df_total.iterrows():
        exact_best = df_total.iloc[idx, 1]
        alns_best = df_total.iloc[idx, 7]
        exact_gap = round((alns_best - exact_best) / exact_best, 4)
        exact_gaps.append(exact_gap)
    
    exact_gap_col = pd.Series(exact_gaps, dtype='float64')    
    df_total['solver gap'] = exact_gap_col
    df_total.round(3)
    return df_total

## ALNS baseline

In [155]:
if generate_df:
    run_baseline_name = f'{run_number}/baseline/'
    run_baseline_df = generate_run_df_alns(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group_alns(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

5-7-1-1 DEVIATES IN SIMULATIONS!


In [156]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,gap
0,5-5-1-1,2519.880,2519.880,2519.880,2852.857,0.727,0.705,0.717,5000,5000,5000.0,4.200,1.600,0.0,0.000,0.0,5,0,0,0,0.000
1,5-5-1-2,2214.210,2214.210,2214.210,2222.835,0.721,0.681,0.701,5000,5000,5000.0,0.200,0.200,0.0,0.000,0.0,1,0,0,4,0.000
2,5-6-1-1,1939.590,1939.590,1939.590,1939.590,0.891,0.847,0.868,5000,5000,5000.0,0.000,0.000,0.0,0.000,0.0,0,0,0,5,0.000
3,5-7-1-1,2709.652,2709.652,2709.652,2896.620,1.304,1.278,1.291,5000,5000,5000.0,21.750,1.250,0.0,0.000,0.0,4,0,0,0,0.000
4,5-7-1-2,1704.602,1704.602,1704.602,1713.227,1.163,1.104,1.125,5000,5000,5000.0,1.400,0.400,0.0,0.000,0.0,2,0,0,3,0.000
5,7-8-1-1,2046.907,2046.907,2046.907,2161.261,1.993,1.717,1.834,5000,5000,5000.0,1.800,1.400,0.0,0.000,0.0,3,0,0,2,0.000
6,7-8-1-2,1971.287,1971.287,1971.287,2059.055,1.992,1.702,1.820,5000,5000,5000.0,12.800,1.600,0.0,0.000,0.0,5,0,0,0,0.000
7,7-8-1-3,2080.178,2080.178,2080.178,2222.701,2.032,1.945,1.989,5000,5000,5000.0,3.400,3.400,0.0,0.000,0.0,5,0,0,0,0.000
8,7-9-1-1,2170.395,2170.395,2170.395,2170.395,4.518,4.234,4.399,5000,5000,5000.0,0.200,0.200,0.0,0.000,0.0,1,0,0,4,0.000
9,7-9-1-2,2206.081,2206.081,2206.081,2206.081,2.823,2.068,2.539,5000,5000,5000.0,0.000,0.000,0.0,0.000,0.0,0,0,0,5,0.000


In [157]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

,instance_group,best obj,worst obj,obj,gap,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.587,2217.587,2217.587,0.000,2325.026,0.961,0.923,0.940,5000.0,5000.0,5000.0,5.510,0.690,0.0,0.000,0.0,2.4,0.0,0.0,2.400
1,7,2094.970,2094.970,2094.970,0.000,2163.899,2.672,2.333,2.516,5000.0,5000.0,5000.0,3.640,1.320,0.0,0.000,0.0,2.8,0.0,0.0,2.200
2,9,5627.549,5627.549,5627.549,0.000,13438.601,4.838,4.147,4.487,5000.0,5000.0,5000.0,51.480,4.920,0.0,0.000,0.0,5.0,0.0,0.0,0.000
3,11,3517.537,3517.537,3517.537,0.000,4424.669,19.857,17.520,18.897,5000.0,5000.0,5000.0,147.760,6.440,0.0,0.000,0.0,5.0,0.0,0.0,0.000
4,13,3970.515,3984.786,3973.369,0.001,6163.880,34.754,30.226,32.340,5000.0,5000.0,5000.0,261.520,8.800,0.0,0.000,0.0,5.0,0.0,0.0,0.000
5,15,8375.449,8429.596,8404.261,0.002,13143.243,39.175,34.654,37.228,5000.0,5000.0,5000.0,1012.400,12.480,0.0,0.006,0.0,5.0,0.0,0.0,0.000
6,17,4967.114,4992.016,4974.691,0.002,6487.388,61.708,52.478,56.720,5000.0,5000.0,5000.0,1096.040,10.720,0.0,0.000,0.0,5.0,0.0,0.0,0.000
7,19,5153.527,5177.385,5164.137,0.002,8906.440,76.459,66.203,70.864,5000.0,5000.0,5000.0,1107.840,13.720,0.0,0.000,0.0,5.0,0.0,0.0,0.000
8,21,9301.246,9358.733,9322.394,0.002,15335.511,93.190,77.617,84.749,5000.0,5000.0,5000.0,1828.720,16.680,0.0,0.000,0.0,5.0,0.0,0.0,0.000
9,23,5967.570,6032.840,6001.004,0.006,8773.028,139.249,123.065,131.681,5000.0,5000.0,5000.0,1421.880,13.680,0.0,0.000,0.0,5.0,0.0,0.0,0.000


## Sequential ALNS

In [158]:
if generate_df:
    run_sequential_name = 'fifth/sequential/'
    run_sequential_df = generate_run_df_alns(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group_alns(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [159]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,gap
0,5-5-1-1,2519.880,2519.880,2519.880,2815.743,0.688,0.660,0.670,5000,5000,5000.0,4.00,1.80,0.000,0.000,0.0,5,0,0,0,0.000
1,5-5-1-2,2214.210,2214.210,2214.210,2240.085,0.650,0.626,0.636,5000,5000,5000.0,0.60,0.60,0.000,0.000,0.0,3,0,0,2,0.000
2,5-6-1-1,1939.590,1939.590,1939.590,1939.590,0.832,0.780,0.800,5000,5000,5000.0,0.00,0.00,0.000,0.000,0.0,0,0,0,5,0.000
3,5-7-1-1,2709.652,2709.652,2709.652,2896.620,1.298,1.181,1.235,5000,5000,5000.0,32.80,1.20,0.000,0.000,0.0,5,0,0,0,0.000
4,5-7-1-2,1704.602,1704.602,1704.602,1721.852,1.095,1.062,1.081,5000,5000,5000.0,3.00,1.00,0.000,0.000,0.0,4,0,0,1,0.000
5,7-8-1-1,2046.907,2046.907,2046.907,2123.143,2.019,1.803,1.875,5000,5000,5000.0,1.80,1.40,0.000,0.000,0.0,2,0,0,3,0.000
6,7-8-1-2,1971.287,1971.287,1971.287,2076.305,1.966,1.755,1.869,5000,5000,5000.0,13.20,2.80,0.000,0.000,0.0,5,0,0,0,0.000
7,7-8-1-3,2080.178,2080.178,2080.178,2297.083,1.950,1.880,1.912,5000,5000,5000.0,4.20,2.60,0.000,0.000,0.0,5,0,0,0,0.000
8,7-9-1-1,2170.395,2170.395,2170.395,2170.395,5.248,5.132,5.206,5000,5000,5000.0,0.20,0.40,0.000,0.000,0.0,2,0,0,3,0.000
9,7-9-1-2,2206.081,2206.081,2206.081,2206.081,3.239,2.353,2.831,5000,5000,5000.0,0.00,0.00,0.000,0.000,0.0,0,0,0,5,0.000


In [160]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

,instance_group,best obj,worst obj,obj,gap,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.587,2217.587,2217.587,0.000,2322.778,0.913,0.862,0.884,5000.0,5000.0,5000.0,8.08,0.92,0.000,0.000,0.0,3.400,0.0,0.0,1.600
1,7,2094.970,2094.970,2094.970,0.000,2174.601,2.884,2.585,2.739,5000.0,5000.0,5000.0,3.88,1.44,0.000,0.000,0.0,2.800,0.0,0.0,2.200
2,9,5627.549,5627.549,5627.549,0.000,13357.220,6.203,5.257,5.773,5000.0,5000.0,5000.0,73.00,5.52,0.000,0.000,0.0,5.000,0.0,0.0,0.000
3,11,3517.537,3517.537,3517.537,0.000,4350.350,28.212,24.353,26.295,5000.0,5000.0,5000.0,121.12,7.76,0.000,0.000,0.0,5.000,0.0,0.0,0.000
4,13,3970.515,3984.786,3973.369,0.001,6210.419,54.872,47.241,50.780,5000.0,5000.0,5000.0,331.88,8.96,0.000,0.000,0.0,5.000,0.0,0.0,0.000
5,15,8375.449,8399.617,8384.454,0.001,13136.445,65.551,49.441,57.221,5000.0,5000.0,5000.0,773.56,12.68,0.080,0.025,0.0,5.000,0.0,0.0,0.000
6,17,4966.142,4987.203,4976.708,0.002,7082.589,99.462,79.679,89.558,5000.0,5000.0,5000.0,878.92,9.80,0.000,0.000,0.0,5.000,0.0,0.0,0.000
7,19,5153.527,5187.338,5165.254,0.002,8275.448,125.746,102.664,111.718,5000.0,5000.0,5000.0,1028.40,13.56,0.000,0.000,0.0,5.000,0.0,0.0,0.000
8,21,9307.827,9368.882,9326.248,0.002,15350.628,151.762,118.314,137.303,5000.0,5000.0,5000.0,1438.48,16.76,0.000,0.000,0.0,5.000,0.0,0.0,0.000
9,23,5959.386,6038.433,5983.775,0.004,8825.868,229.714,199.817,215.472,5000.0,5000.0,5000.0,1591.24,13.92,0.000,0.000,0.0,5.000,0.0,0.0,0.000


## LNS

In [161]:
if generate_df:
    run_lns_name = f'{run_number}/lns/'
    run_lns_df = generate_run_df_alns(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group_alns(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

9-9-1-2 DEVIATES IN SIMULATIONS!


In [162]:
run_lns_df = load_df('lns.pkl')
run_lns_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,gap
0,5-5-1-1,2519.880,2519.880,2519.880,2852.857,0.752,0.688,0.730,5000,5000,5000.0,7.400,2.00,0.0,0.000,0.0,5,0,0,0,0.000
1,5-5-1-2,2214.210,2214.210,2214.210,2240.085,0.701,0.674,0.686,5000,5000,5000.0,2.200,0.60,0.0,0.000,0.0,3,0,0,2,0.000
2,5-6-1-1,1939.590,1939.590,1939.590,1939.590,0.880,0.828,0.859,5000,5000,5000.0,0.000,0.00,0.0,0.000,0.0,0,0,0,5,0.000
3,5-7-1-1,2709.652,2709.652,2709.652,2896.620,1.298,1.237,1.262,5000,5000,5000.0,14.800,1.20,0.0,0.000,0.0,5,0,0,0,0.000
4,5-7-1-2,1704.602,1704.602,1704.602,1713.227,1.135,1.097,1.113,5000,5000,5000.0,1.800,0.40,0.0,0.000,0.0,2,0,0,3,0.000
5,7-8-1-1,2046.907,2046.907,2046.907,2123.143,1.936,1.774,1.855,5000,5000,5000.0,2.000,1.20,0.0,0.000,0.0,2,0,0,3,0.000
6,7-8-1-2,1971.287,1971.287,1971.287,2047.515,1.803,1.593,1.693,5000,5000,5000.0,8.000,1.80,0.0,0.000,0.0,5,0,0,0,0.000
7,7-8-1-3,2080.178,2080.178,2080.178,2259.892,2.027,1.812,1.960,5000,5000,5000.0,9.400,3.00,0.0,0.000,0.0,5,0,0,0,0.000
8,7-9-1-1,2170.395,2170.395,2170.395,2170.395,4.501,4.067,4.331,5000,5000,5000.0,0.000,0.00,0.0,0.000,0.0,0,0,0,5,0.000
9,7-9-1-2,2206.081,2206.081,2206.081,2206.081,2.832,2.162,2.524,5000,5000,5000.0,0.000,0.00,0.0,0.000,0.0,0,0,0,5,0.000


In [163]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

,instance_group,best obj,worst obj,obj,gap,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.587,2217.587,2217.587,0.000,2328.476,0.953,0.905,0.930,5000.0,5000.0,5000.0,5.240,0.84,0.0,0.000,0.0,3.0,0.0,0.0,2.000
1,7,2094.970,2094.970,2094.970,0.000,2161.405,2.620,2.282,2.473,5000.0,5000.0,5000.0,3.880,1.20,0.0,0.000,0.0,2.4,0.0,0.0,2.600
2,9,5627.549,5627.549,5627.549,0.000,13434.316,5.044,4.110,4.505,5000.0,5000.0,5000.0,64.010,4.92,0.0,0.000,0.0,4.8,0.0,0.0,0.000
3,11,3517.537,3517.537,3517.537,0.000,4352.075,20.047,17.539,18.624,5000.0,5000.0,5000.0,151.800,7.28,0.0,0.000,0.0,5.0,0.0,0.0,0.000
4,13,3970.515,3974.593,3971.330,0.000,5964.085,34.829,30.074,32.135,5000.0,5000.0,5000.0,423.040,9.52,0.0,0.000,0.0,5.0,0.0,0.0,0.000
5,15,8377.122,8435.094,8405.221,0.002,13368.764,39.272,34.036,36.541,5000.0,5000.0,5000.0,1147.240,14.32,0.0,0.006,0.0,5.0,0.0,0.0,0.000
6,17,4968.559,5002.444,4978.507,0.002,7118.488,61.599,52.667,57.219,5000.0,5000.0,5000.0,976.680,11.08,0.0,0.000,0.0,5.0,0.0,0.0,0.000
7,19,5153.527,5185.788,5168.693,0.003,8630.828,79.251,62.547,71.421,5000.0,5000.0,5000.0,1158.240,13.04,0.0,0.000,0.0,5.0,0.0,0.0,0.000
8,21,9262.159,9388.044,9321.177,0.005,15371.046,89.480,74.979,82.631,5000.0,5000.0,5000.0,1624.840,15.44,0.0,0.000,0.0,5.0,0.0,0.0,0.000
9,23,5959.560,6043.360,5992.712,0.006,8854.713,138.342,117.610,129.258,5000.0,5000.0,5000.0,1555.760,14.56,0.0,0.000,0.0,5.0,0.0,0.0,0.000


## ALNS + local search

In [164]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    run_ls_df = generate_run_df_alns(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group_alns(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

In [165]:
run_ls_df = load_df('ls.pkl')
run_ls_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,gap
0,5-5-1-1,2519.880,2519.880,2519.880,2815.743,1.235,1.171,1.197,5000,5000,5000.000,6.400,0.600,1.600,0.137,0.0,2,3,0,0,0.000
1,5-5-1-2,2214.210,2214.210,2214.210,2240.085,1.193,1.068,1.109,5000,5000,5000.000,1.200,0.200,0.400,0.019,0.0,1,2,0,2,0.000
2,5-6-1-1,1939.590,1939.590,1939.590,1939.590,1.329,1.269,1.303,5000,5000,5000.000,0.000,0.000,0.000,0.000,0.0,0,0,0,5,0.000
3,5-7-1-1,2709.652,2709.652,2709.652,2896.620,2.020,1.884,1.965,5000,5000,5000.000,2.000,0.000,1.400,0.118,0.0,0,5,0,0,0.000
4,5-7-1-2,1704.602,1704.602,1704.602,1713.227,1.673,1.588,1.629,5000,5000,5000.000,0.000,0.000,0.400,0.012,0.0,0,2,0,3,0.000
5,7-8-1-1,2046.907,2046.907,2046.907,2161.261,3.488,3.159,3.359,5000,5000,5000.000,1.200,0.400,1.200,0.065,0.0,1,2,0,2,0.000
6,7-8-1-2,1971.287,1971.287,1971.287,2028.807,3.564,2.834,3.160,5000,5000,5000.000,4.600,0.400,1.200,0.043,0.0,2,2,0,1,0.000
7,7-8-1-3,2080.178,2080.178,2080.178,2259.892,3.586,3.340,3.429,5000,5000,5000.000,2.600,1.000,1.600,0.129,0.0,2,3,0,0,0.000
8,7-9-1-1,2170.395,2170.395,2170.395,2170.395,6.494,5.982,6.295,5000,5000,5000.000,0.000,0.000,0.200,0.029,0.0,0,1,0,4,0.000
9,7-9-1-2,2206.081,2206.081,2206.081,2206.081,4.533,3.720,4.289,5000,5000,5000.000,0.000,0.000,0.000,0.016,0.0,0,0,0,5,0.000


In [166]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

,instance_group,best obj,worst obj,obj,gap,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.587,2217.587,2217.587,0.000,2321.053,1.490,1.396,1.441,5000.0,5000.000,5000.000,1.920,0.160,0.760,0.057,0.0,0.60,2.400,0.0,2.000
1,7,2094.970,2094.970,2094.970,0.000,2165.287,4.333,3.807,4.106,5000.0,5000.000,5000.000,1.680,0.360,0.840,0.056,0.0,1.00,1.600,0.0,2.400
2,9,5627.549,5627.549,5627.549,0.000,13458.526,8.318,6.769,7.540,5000.0,5000.000,5000.000,100.840,1.480,3.200,0.220,0.0,2.80,2.200,0.0,0.000
3,11,3517.537,3517.537,3517.537,0.000,4480.290,44.558,39.621,41.965,5000.0,5000.000,5000.000,65.080,0.880,5.080,0.126,0.0,1.80,3.200,0.0,0.000
4,13,3970.515,3970.515,3970.515,0.000,6098.740,77.633,67.049,72.164,5000.0,5000.000,5000.000,109.200,0.600,6.240,0.145,0.0,1.00,4.000,0.0,0.000
5,15,8375.449,8385.372,8377.433,0.000,13310.770,82.175,70.279,76.074,5000.0,5000.000,5000.000,938.760,0.840,8.400,0.166,0.0,0.80,4.200,0.0,0.000
6,17,4966.142,4969.530,4966.820,0.000,7149.787,162.210,135.053,149.536,5000.0,5000.000,5000.000,896.640,1.000,9.400,0.161,0.0,0.80,4.200,0.0,0.000
7,19,5153.527,5159.335,5155.206,0.000,8898.013,198.963,180.729,189.572,5000.0,5000.000,5000.000,990.400,0.720,10.840,0.171,0.0,1.20,3.800,0.0,0.000
8,21,9248.071,9326.905,9292.684,0.003,15496.872,189.306,162.536,174.167,5000.0,5000.000,5000.000,2293.160,1.200,12.840,0.166,0.0,1.80,3.200,0.0,0.000
9,23,5962.993,5987.220,5968.665,0.001,9044.719,355.671,308.091,333.033,5000.0,5000.000,5000.000,1574.680,0.880,13.280,0.134,0.0,1.20,3.800,0.0,0.000


## ALNS + set partitioning

In [167]:
if generate_df: 
    run_sp_name = f'{run_number}/sp/'
    run_sp_df = generate_run_df_alns(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group_alns(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

19-21-3-2 DEVIATES IN SIMULATIONS!


In [168]:
run_sp_df = load_df('sp.pkl')
run_sp_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,gap
0,5-5-1-1,2519.880,2519.880,2519.880,2834.300,0.791,0.774,0.784,5000,5000,5000.0,14.200,2.000,0.000,0.000,0.00,5,0,0,0,0.000
1,5-5-1-2,2214.210,2214.210,2214.210,2389.125,0.775,0.727,0.749,5000,5000,5000.0,1.000,1.000,0.000,0.000,0.00,4,0,0,1,0.000
2,5-6-1-1,1939.590,1939.590,1939.590,1939.590,0.947,0.895,0.922,5000,5000,5000.0,0.000,0.000,0.000,0.000,0.00,0,0,0,5,0.000
3,5-7-1-1,2709.652,2709.652,2709.652,2896.620,1.460,1.341,1.384,5000,5000,5000.0,14.600,1.200,0.000,0.000,0.00,5,0,0,0,0.000
4,5-7-1-2,1704.602,1704.602,1704.602,1717.539,1.368,1.283,1.321,5000,5000,5000.0,2.200,1.000,0.000,0.000,0.00,3,0,0,2,0.000
5,7-8-1-1,2046.907,2046.907,2046.907,2123.143,2.176,1.916,2.052,5000,5000,5000.0,1.600,1.200,0.000,0.000,0.00,2,0,0,3,0.000
6,7-8-1-2,1971.287,1971.287,1971.287,2046.117,2.242,1.843,1.990,5000,5000,5000.0,9.400,1.400,0.000,0.000,0.00,4,0,0,1,0.000
7,7-8-1-3,2080.178,2080.178,2080.178,2259.892,2.223,2.115,2.155,5000,5000,5000.0,3.600,2.800,0.000,0.000,0.00,5,0,0,0,0.000
8,7-9-1-1,2170.395,2170.395,2170.395,2170.395,4.923,4.691,4.800,5000,5000,5000.0,0.000,0.000,0.000,0.000,0.00,0,0,0,5,0.000
9,7-9-1-2,2206.081,2206.081,2206.081,2206.081,3.205,2.355,2.659,5000,5000,5000.0,0.000,0.000,0.000,0.000,0.00,0,0,0,5,0.000


In [169]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

,instance_group,best obj,worst obj,obj,gap,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.587,2217.587,2217.587,0.000,2355.435,1.068,1.004,1.032,5000.0,5000.0,5000.0,6.400,1.040,0.000,0.000,0.00,3.400,0.0,0.0,1.600
1,7,2094.970,2094.970,2094.970,0.000,2161.126,2.954,2.584,2.731,5000.0,5000.0,5000.0,2.920,1.080,0.000,0.000,0.00,2.200,0.0,0.0,2.800
2,9,5627.549,5627.549,5627.549,0.000,13445.305,5.909,5.113,5.504,5000.0,5000.0,5000.0,67.560,5.400,0.000,0.000,0.16,5.000,0.0,0.0,0.000
3,11,3517.537,3517.537,3517.537,0.000,4426.117,25.106,21.442,23.186,5000.0,5000.0,5000.0,108.560,7.160,0.000,0.000,0.04,5.000,0.0,0.0,0.000
4,13,3970.515,3984.786,3973.369,0.001,6028.905,50.292,42.919,47.125,5000.0,5000.0,5000.0,238.000,8.680,0.000,0.000,0.08,5.000,0.0,0.0,0.000
5,15,8375.449,8392.201,8386.750,0.001,13182.776,63.925,49.510,56.699,5000.0,5000.0,5000.0,598.280,10.240,0.080,0.017,1.12,5.000,0.0,0.0,0.000
6,17,4966.142,4987.539,4972.075,0.001,6963.270,105.024,82.662,95.515,5000.0,5000.0,5000.0,629.600,11.920,0.000,0.000,1.44,5.000,0.0,0.0,0.000
7,19,5153.527,5171.619,5159.202,0.001,8620.451,145.987,120.894,131.968,5000.0,5000.0,5000.0,754.570,13.600,0.000,0.000,1.64,4.800,0.0,0.0,0.000
8,21,9255.841,9318.897,9286.044,0.002,15486.476,172.279,147.416,161.238,5000.0,5000.0,5000.0,1650.520,14.880,0.000,0.002,2.44,5.000,0.0,0.0,0.000
9,23,5959.386,5998.931,5971.851,0.002,8639.918,248.075,208.014,225.655,5000.0,5000.0,5000.0,1668.280,14.560,0.000,0.000,2.24,5.000,0.0,0.0,0.000


## ALNS + local search + set partitioning

In [170]:
if generate_df:
    run_lssp_name = f'{run_number}/lssp/'
    run_lssp_df = generate_run_df_alns(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group_alns(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

9-10-1-1 DEVIATES IN SIMULATIONS!


In [171]:
run_lssp_df = load_df('lssp.pkl')
run_lssp_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,gap
0,5-5-1-1,2519.880,2519.880,2519.880,2834.300,1.351,1.268,1.310,5000,5000,5000.00,0.600,0.400,1.000,0.137,0.000,2,3,0,0,0.000
1,5-5-1-2,2214.210,2214.210,2214.210,2240.085,1.201,1.127,1.172,5000,5000,5000.00,0.000,0.200,0.400,0.019,0.000,1,2,0,2,0.000
2,5-6-1-1,1939.590,1939.590,1939.590,1939.590,1.500,1.374,1.417,5000,5000,5000.00,0.000,0.000,0.000,0.000,0.000,0,0,0,5,0.000
3,5-7-1-1,2709.652,2709.652,2709.652,2896.620,2.144,2.035,2.060,5000,5000,5000.00,0.400,0.000,1.000,0.144,0.000,0,5,0,0,0.000
4,5-7-1-2,1704.602,1704.602,1704.602,1713.227,1.807,1.770,1.790,5000,5000,5000.00,0.400,0.000,0.400,0.012,0.000,0,2,0,3,0.000
5,7-8-1-1,2046.907,2046.907,2046.907,2085.025,3.951,3.509,3.657,5000,5000,5000.00,0.600,0.200,0.200,0.065,0.000,0,1,0,4,0.000
6,7-8-1-2,1971.287,1971.287,1971.287,2015.870,3.451,2.983,3.143,5000,5000,5000.00,4.200,0.600,0.800,0.033,0.000,3,0,0,2,0.000
7,7-8-1-3,2080.178,2080.178,2080.178,2222.701,3.863,3.504,3.770,5000,5000,5000.00,2.000,0.600,1.800,0.121,0.000,1,4,0,0,0.000
8,7-9-1-1,2170.395,2170.395,2170.395,2170.395,7.259,6.764,7.037,5000,5000,5000.00,0.000,0.000,0.200,0.029,0.000,0,1,0,4,0.000
9,7-9-1-2,2206.081,2206.081,2206.081,2206.081,4.968,4.565,4.779,5000,5000,5000.00,0.000,0.000,0.000,0.019,0.000,0,0,0,5,0.000


In [172]:
run_lssp_agg_df = load_df('lssp_agg.pkl')
run_lssp_agg_df

,instance_group,best obj,worst obj,obj,gap,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.587,2217.587,2217.587,0.000,2324.764,1.601,1.515,1.550,5000.0,5000.000,5000.00,0.280,0.120,0.560,0.062,0.000,0.6,2.400,0.0,2.000
1,7,2094.970,2094.970,2094.970,0.000,2140.014,4.698,4.265,4.477,5000.0,5000.000,5000.00,1.360,0.280,0.600,0.053,0.000,0.8,1.200,0.0,3.000
2,9,5627.549,5627.549,5627.549,0.000,13335.930,8.639,7.825,8.268,5000.0,5000.000,5000.00,53.890,1.540,3.140,0.213,0.040,3.4,1.400,0.0,0.000
3,11,3517.537,3517.537,3517.537,0.000,4367.323,54.731,45.643,50.258,5000.0,5000.000,5000.00,63.320,0.640,4.680,0.128,0.040,1.4,3.600,0.0,0.000
4,13,3970.515,3970.515,3970.515,0.000,6182.491,88.465,79.849,84.028,5000.0,5000.000,5000.00,115.240,0.880,5.640,0.173,0.040,1.0,4.000,0.0,0.000
5,15,8375.449,8375.449,8375.449,0.000,13152.487,97.318,84.598,91.100,5000.0,5000.000,5000.00,369.360,1.160,8.560,0.177,0.840,1.0,4.000,0.0,0.000
6,17,4966.142,4966.142,4966.142,0.000,7152.594,184.166,151.907,167.719,5000.0,5000.000,5000.00,621.240,1.480,6.880,0.165,0.720,2.6,2.400,0.0,0.000
7,19,5153.527,5153.527,5153.527,0.000,8712.660,241.606,211.089,224.294,5000.0,5000.000,5000.00,878.200,2.000,9.880,0.170,1.200,1.6,3.400,0.0,0.000
8,21,9251.875,9297.583,9265.805,0.001,15413.953,213.074,172.627,195.248,5000.0,5000.000,5000.00,1381.720,2.840,9.680,0.178,1.800,2.2,2.800,0.0,0.000
9,23,5959.386,5963.698,5962.097,0.000,8822.954,392.783,332.663,363.911,5000.0,5000.000,5000.00,1188.480,1.240,11.440,0.133,0.840,1.6,3.400,0.0,0.000


## Exact solver 3600

In [173]:
if generate_df:
    run_exact_3600_name = '3600/results/'
    run_exact_3600_df = generate_run_df_exact(run_exact_3600_name)
    run_exact_3600_agg_df = aggregate_df_by_instance_group_exact(run_exact_3600_df)

    run_exact_3600_file_name = f'dataframes/performance/exact_3600.pkl'
    run_exact_3600_agg_file_name = f'dataframes/performance/exact_3600_agg.pkl'
    run_exact_3600_df.to_pickle(run_exact_3600_file_name)
    run_exact_3600_agg_df.to_pickle(run_exact_3600_agg_file_name)

In [174]:
run_exact_3600_df = load_df('exact_3600.pkl')
run_exact_3600_df

,instance,obj,gap,preproc,time,variables
0,5-5-1-1,2519.880,0.000,1.466,1.276,24856
1,5-5-1-2,2214.210,0.000,1.581,3.726,36406
2,5-6-1-1,1939.590,0.000,1.770,4.284,40610
3,5-7-1-1,2709.652,0.000,2.268,4.182,35090
4,5-7-1-2,1704.602,0.000,2.627,16.189,64346
5,7-8-1-1,2046.907,0.000,3.494,24.273,71178
6,7-8-1-2,1971.287,0.000,3.576,65.171,84072
7,7-8-1-3,2080.177,0.000,3.043,9.720,47988
8,7-9-1-1,2170.395,0.000,3.481,26.712,80958
9,7-9-1-2,2206.081,0.000,3.683,38.585,85362


In [175]:
run_exact_3600_agg_df = load_df('exact_3600_agg.pkl')
run_exact_3600_agg_df

,instance_group,obj,gap,preproc,time,variables
0,5,2217.587,0.000,1.942,5.931,40261.600
1,7,2094.969,0.000,3.455,32.892,73911.600
2,9,5627.549,0.000,5.674,48.502,124834.400
3,11,3517.537,0.026,13.823,2417.191,309944.400
4,13,3994.678,0.079,20.265,3567.688,447577.200
5,15,12477.863,0.258,27.265,3600.441,606885.000
6,17,8567.211,0.610,44.381,3601.990,984296.000
7,19,8988.366,0.611,52.334,3601.088,1136150.400
8,21,13325.404,0.445,65.790,3600.658,1435097.600
9,23,801846.912,80.157,98.278,1200.031,2588391.000


## Exact solver 600

In [176]:
if generate_df:
    run_exact_600_name = '600/results/'
    run_exact_600_df = generate_run_df_exact(run_exact_600_name)
    run_exact_600_agg_df = aggregate_df_by_instance_group_exact(run_exact_600_df)

    run_exact_600_file_name = f'dataframes/performance/exact_600.pkl'
    run_exact_600_agg_file_name = f'dataframes/performance/exact_600_agg.pkl'
    run_exact_600_df.to_pickle(run_exact_600_file_name)
    run_exact_600_agg_df.to_pickle(run_exact_600_agg_file_name)

In [177]:
run_exact_600_df = load_df('exact_600.pkl')
run_exact_600_df

,instance,obj,gap,preproc,time,variables
0,5-5-1-1,2519.880,0.000,1.443,1.256,24856
1,5-5-1-2,2214.210,0.000,1.564,3.721,36406
2,5-6-1-1,1939.590,0.000,1.749,4.248,40610
3,5-7-1-1,2709.652,0.000,2.235,4.138,35090
4,5-7-1-2,1704.602,0.000,2.662,16.112,64346
5,7-8-1-1,2046.907,0.000,3.470,24.004,71178
6,7-8-1-2,1971.287,0.000,3.552,65.084,84072
7,7-8-1-3,2080.177,0.000,2.997,9.723,47988
8,7-9-1-1,2170.395,0.000,3.528,26.477,80958
9,7-9-1-2,2206.081,0.000,3.722,39.063,85362


In [178]:
run_exact_600_agg_df = load_df('exact_600_agg.pkl')
run_exact_600_agg_df

,instance_group,obj,gap,preproc,time,variables
0,5,2217.587,0.000,1.931,5.895,40261.60
1,7,2094.969,0.000,3.454,32.870,73911.60
2,9,5627.549,0.000,5.642,48.607,124834.40
3,11,5369.449,0.272,13.845,554.443,309944.40
4,13,10025.767,0.654,20.274,600.324,447577.20
5,15,208617.891,20.237,40.306,600.041,606885.00
6,17,404884.820,40.412,63.789,600.049,980112.00
7,19,405441.420,40.494,68.967,600.151,1125112.00
8,21,801358.178,80.074,92.486,600.108,1417245.60
9,23,1000000.000,100.000,100.000,600.000,2107146.00


## Parallel vs. sequential heuristics

In [179]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df], True)
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df], True)

In [180]:
baseline_sequential_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,gap
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,5-5-1-1,2519.9,0.7,5000.0,1.8,0.0,0.0,0.0,5,0,0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,5-5-1-2,2214.2,0.6,5000.0,0.6,0.0,0.0,0.0,3,0,0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,5-6-1-1,1939.6,0.8,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,5-7-1-1,2709.7,1.2,5000.0,1.2,0.0,0.0,0.0,5,0,0,0.0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,5-7-1-2,1704.6,1.1,5000.0,1.0,0.0,0.0,0.0,4,0,0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,7-8-1-1,2046.9,1.9,5000.0,1.4,0.0,0.0,0.0,2,0,0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,7-8-1-2,1971.3,1.9,5000.0,2.8,0.0,0.0,0.0,5,0,0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,7-8-1-3,2080.2,1.9,5000.0,2.6,0.0,0.0,0.0,5,0,0,0.0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,7-9-1-1,2170.4,5.2,5000.0,0.4,0.0,0.0,0.0,2,0,0,0.0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,7-9-1-2,2206.1,2.8,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0


In [181]:
baseline_sequential_agg_df

,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,crfb
0,5,2217.6,0.0,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,5,2217.6,0.0,0.9,5000.0,0.9,0.0,0.0,0.0,3.4,0.0,1.6
1,7,2095.0,0.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,7,2095.0,0.0,2.7,5000.0,1.4,0.0,0.0,0.0,2.8,0.0,2.2
2,9,5627.5,0.0,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,9,5627.5,0.0,5.8,5000.0,5.5,0.0,0.0,0.0,5.0,0.0,0.0
3,11,3517.5,0.0,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,11,3517.5,0.0,26.3,5000.0,7.8,0.0,0.0,0.0,5.0,0.0,0.0
4,13,3973.4,0.0,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,13,3973.4,0.0,50.8,5000.0,9.0,0.0,0.0,0.0,5.0,0.0,0.0
5,15,8404.3,0.0,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,15,8384.5,0.0,57.2,5000.0,12.7,0.1,0.0,0.0,5.0,0.0,0.0
6,17,4974.7,0.0,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,17,4976.7,0.0,89.6,5000.0,9.8,0.0,0.0,0.0,5.0,0.0,0.0
7,19,5164.1,0.0,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,19,5165.3,0.0,111.7,5000.0,13.6,0.0,0.0,0.0,5.0,0.0,0.0
8,21,9322.4,0.0,84.7,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,21,9326.2,0.0,137.3,5000.0,16.8,0.0,0.0,0.0,5.0,0.0,0.0
9,23,6001.0,0.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,23,5983.8,0.0,215.5,5000.0,13.9,0.0,0.0,0.0,5.0,0.0,0.0


## ALNS vs. LNS

In [182]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df], True)
baseline_lns_agg_df = merge_dfs([run_baseline_agg_df, run_lns_agg_df], True)

In [183]:
baseline_lns_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,gap
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,5-5-1-1,2519.9,0.7,5000.0,2.0,0.0,0.0,0.0,5,0,0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,5-5-1-2,2214.2,0.7,5000.0,0.6,0.0,0.0,0.0,3,0,0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,5,0,0,0.0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,7-8-1-1,2046.9,1.9,5000.0,1.2,0.0,0.0,0.0,2,0,0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,7-8-1-2,1971.3,1.7,5000.0,1.8,0.0,0.0,0.0,5,0,0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,7-8-1-3,2080.2,2.0,5000.0,3.0,0.0,0.0,0.0,5,0,0,0.0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,7-9-1-1,2170.4,4.3,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0


In [184]:
baseline_lns_agg_df

,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,crfb
0,5,2217.6,0.0,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,5,2217.6,0.0,0.9,5000.0,0.8,0.0,0.0,0.0,3.0,0.0,2.0
1,7,2095.0,0.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,7,2095.0,0.0,2.5,5000.0,1.2,0.0,0.0,0.0,2.4,0.0,2.6
2,9,5627.5,0.0,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,9,5627.5,0.0,4.5,5000.0,4.9,0.0,0.0,0.0,4.8,0.0,0.0
3,11,3517.5,0.0,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,11,3517.5,0.0,18.6,5000.0,7.3,0.0,0.0,0.0,5.0,0.0,0.0
4,13,3973.4,0.0,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,13,3971.3,0.0,32.1,5000.0,9.5,0.0,0.0,0.0,5.0,0.0,0.0
5,15,8404.3,0.0,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,15,8405.2,0.0,36.5,5000.0,14.3,0.0,0.0,0.0,5.0,0.0,0.0
6,17,4974.7,0.0,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,17,4978.5,0.0,57.2,5000.0,11.1,0.0,0.0,0.0,5.0,0.0,0.0
7,19,5164.1,0.0,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,19,5168.7,0.0,71.4,5000.0,13.0,0.0,0.0,0.0,5.0,0.0,0.0
8,21,9322.4,0.0,84.7,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,21,9321.2,0.0,82.6,5000.0,15.4,0.0,0.0,0.0,5.0,0.0,0.0
9,23,6001.0,0.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,23,5992.7,0.0,129.3,5000.0,14.6,0.0,0.0,0.0,5.0,0.0,0.0


## ALNS vs. ALNS + local search

In [185]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df], True)
baseline_ls_agg_df = merge_dfs([run_baseline_agg_df, run_ls_agg_df], True)

In [186]:
baseline_ls_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,gap
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,5-5-1-1,2519.9,1.2,5000.0,0.6,1.6,0.1,0.0,2,3,0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,5-5-1-2,2214.2,1.1,5000.0,0.2,0.4,0.0,0.0,1,2,0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,5-6-1-1,1939.6,1.3,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,5-7-1-1,2709.7,2.0,5000.0,0.0,1.4,0.1,0.0,0,5,0,0.0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,5-7-1-2,1704.6,1.6,5000.0,0.0,0.4,0.0,0.0,0,2,0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,7-8-1-1,2046.9,3.4,5000.0,0.4,1.2,0.1,0.0,1,2,0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,7-8-1-2,1971.3,3.2,5000.0,0.4,1.2,0.0,0.0,2,2,0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,7-8-1-3,2080.2,3.4,5000.0,1.0,1.6,0.1,0.0,2,3,0,0.0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,7-9-1-1,2170.4,6.3,5000.0,0.0,0.2,0.0,0.0,0,1,0,0.0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,7-9-1-2,2206.1,4.3,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0


In [187]:
baseline_ls_agg_df

,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,crfb
0,5,2217.6,0.0,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,5,2217.6,0.0,1.4,5000.0,0.2,0.8,0.1,0.0,0.6,2.4,2.0
1,7,2095.0,0.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,7,2095.0,0.0,4.1,5000.0,0.4,0.8,0.1,0.0,1.0,1.6,2.4
2,9,5627.5,0.0,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,9,5627.5,0.0,7.5,5000.0,1.5,3.2,0.2,0.0,2.8,2.2,0.0
3,11,3517.5,0.0,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,11,3517.5,0.0,42.0,5000.0,0.9,5.1,0.1,0.0,1.8,3.2,0.0
4,13,3973.4,0.0,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,13,3970.5,0.0,72.2,5000.0,0.6,6.2,0.1,0.0,1.0,4.0,0.0
5,15,8404.3,0.0,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,15,8377.4,0.0,76.1,5000.0,0.8,8.4,0.2,0.0,0.8,4.2,0.0
6,17,4974.7,0.0,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,17,4966.8,0.0,149.5,5000.0,1.0,9.4,0.2,0.0,0.8,4.2,0.0
7,19,5164.1,0.0,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,19,5155.2,0.0,189.6,5000.0,0.7,10.8,0.2,0.0,1.2,3.8,0.0
8,21,9322.4,0.0,84.7,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,21,9292.7,0.0,174.2,5000.0,1.2,12.8,0.2,0.0,1.8,3.2,0.0
9,23,6001.0,0.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,23,5968.7,0.0,333.0,5000.0,0.9,13.3,0.1,0.0,1.2,3.8,0.0


## ALNS vs. ALNS + set partitioning

In [188]:
baseline_sp_df = merge_dfs([run_baseline_df, run_sp_df], True)
baseline_sp_agg_df = merge_dfs([run_baseline_agg_df, run_sp_agg_df], True)

In [189]:
baseline_sp_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,gap
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,5-5-1-1,2519.9,0.8,5000.0,2.0,0.0,0.0,0.0,5,0,0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,5-5-1-2,2214.2,0.7,5000.0,1.0,0.0,0.0,0.0,4,0,0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,5-7-1-1,2709.7,1.4,5000.0,1.2,0.0,0.0,0.0,5,0,0,0.0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,5-7-1-2,1704.6,1.3,5000.0,1.0,0.0,0.0,0.0,3,0,0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,7-8-1-1,2046.9,2.1,5000.0,1.2,0.0,0.0,0.0,2,0,0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,7-8-1-2,1971.3,2.0,5000.0,1.4,0.0,0.0,0.0,4,0,0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,7-8-1-3,2080.2,2.2,5000.0,2.8,0.0,0.0,0.0,5,0,0,0.0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,7-9-1-1,2170.4,4.8,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,7-9-1-2,2206.1,2.7,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0


In [190]:
baseline_sp_agg_df

,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,crfb
0,5,2217.6,0.0,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,5,2217.6,0.0,1.0,5000.0,1.0,0.0,0.0,0.0,3.4,0.0,1.6
1,7,2095.0,0.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,7,2095.0,0.0,2.7,5000.0,1.1,0.0,0.0,0.0,2.2,0.0,2.8
2,9,5627.5,0.0,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,9,5627.5,0.0,5.5,5000.0,5.4,0.0,0.0,0.2,5.0,0.0,0.0
3,11,3517.5,0.0,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,11,3517.5,0.0,23.2,5000.0,7.2,0.0,0.0,0.0,5.0,0.0,0.0
4,13,3973.4,0.0,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,13,3973.4,0.0,47.1,5000.0,8.7,0.0,0.0,0.1,5.0,0.0,0.0
5,15,8404.3,0.0,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,15,8386.8,0.0,56.7,5000.0,10.2,0.1,0.0,1.1,5.0,0.0,0.0
6,17,4974.7,0.0,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,17,4972.1,0.0,95.5,5000.0,11.9,0.0,0.0,1.4,5.0,0.0,0.0
7,19,5164.1,0.0,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,19,5159.2,0.0,132.0,5000.0,13.6,0.0,0.0,1.6,4.8,0.0,0.0
8,21,9322.4,0.0,84.7,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,21,9286.0,0.0,161.2,5000.0,14.9,0.0,0.0,2.4,5.0,0.0,0.0
9,23,6001.0,0.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,23,5971.9,0.0,225.7,5000.0,14.6,0.0,0.0,2.2,5.0,0.0,0.0


## ALNS vs. ALNS + local search + set partitioning

In [191]:
baseline_lssp_df = merge_dfs([run_baseline_df, run_lssp_df], True)
baseline_lssp_agg_df = merge_dfs([run_baseline_agg_df, run_lssp_agg_df], True)

In [192]:
baseline_lssp_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,gap
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,5-5-1-1,2519.9,1.3,5000.0,0.4,1.0,0.1,0.0,2,3,0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,5-5-1-2,2214.2,1.2,5000.0,0.2,0.4,0.0,0.0,1,2,0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,5-6-1-1,1939.6,1.4,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,5-7-1-1,2709.7,2.1,5000.0,0.0,1.0,0.1,0.0,0,5,0,0.0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,5-7-1-2,1704.6,1.8,5000.0,0.0,0.4,0.0,0.0,0,2,0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,7-8-1-1,2046.9,3.7,5000.0,0.2,0.2,0.1,0.0,0,1,0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,7-8-1-2,1971.3,3.1,5000.0,0.6,0.8,0.0,0.0,3,0,0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,7-8-1-3,2080.2,3.8,5000.0,0.6,1.8,0.1,0.0,1,4,0,0.0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,7-9-1-1,2170.4,7.0,5000.0,0.0,0.2,0.0,0.0,0,1,0,0.0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,7-9-1-2,2206.1,4.8,5000.0,0.0,0.0,0.0,0.0,0,0,0,0.0


In [193]:
baseline_lssp_agg_df

,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,instance_group,obj,gap,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,crfb
0,5,2217.6,0.0,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,5,2217.6,0.0,1.6,5000.0,0.1,0.6,0.1,0.0,0.6,2.4,2.0
1,7,2095.0,0.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,7,2095.0,0.0,4.5,5000.0,0.3,0.6,0.1,0.0,0.8,1.2,3.0
2,9,5627.5,0.0,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,9,5627.5,0.0,8.3,5000.0,1.5,3.1,0.2,0.0,3.4,1.4,0.0
3,11,3517.5,0.0,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,11,3517.5,0.0,50.3,5000.0,0.6,4.7,0.1,0.0,1.4,3.6,0.0
4,13,3973.4,0.0,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,13,3970.5,0.0,84.0,5000.0,0.9,5.6,0.2,0.0,1.0,4.0,0.0
5,15,8404.3,0.0,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,15,8375.4,0.0,91.1,5000.0,1.2,8.6,0.2,0.8,1.0,4.0,0.0
6,17,4974.7,0.0,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,17,4966.1,0.0,167.7,5000.0,1.5,6.9,0.2,0.7,2.6,2.4,0.0
7,19,5164.1,0.0,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,19,5153.5,0.0,224.3,5000.0,2.0,9.9,0.2,1.2,1.6,3.4,0.0
8,21,9322.4,0.0,84.7,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,21,9265.8,0.0,195.2,5000.0,2.8,9.7,0.2,1.8,2.2,2.8,0.0
9,23,6001.0,0.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,23,5962.1,0.0,363.9,5000.0,1.2,11.4,0.1,0.8,1.6,3.4,0.0


## Best ALNS vs exact solver

In [194]:
exact_alns_df = merge_dfs_exact_alns(run_exact_3600_df, run_exact_600_df, run_lssp_df)
exact_alns_agg_df = merge_dfs_exact_alns(run_exact_3600_agg_df, run_exact_600_agg_df, run_lssp_agg_df)

In [195]:
exact_alns_df

,instance,obj,gap,time,obj,gap,time,obj,time,gap,solver gap
0,5-5-1-1,2519.880,0.000,1.276,2519.880,0.000,1.256,2519.880,1.310,0.000,0.0000
1,5-5-1-2,2214.210,0.000,3.726,2214.210,0.000,3.721,2214.210,1.172,0.000,0.0000
2,5-6-1-1,1939.590,0.000,4.284,1939.590,0.000,4.248,1939.590,1.417,0.000,0.0000
3,5-7-1-1,2709.652,0.000,4.182,2709.652,0.000,4.138,2709.652,2.060,0.000,0.0000
4,5-7-1-2,1704.602,0.000,16.189,1704.602,0.000,16.112,1704.602,1.790,0.000,0.0000
5,7-8-1-1,2046.907,0.000,24.273,2046.907,0.000,24.004,2046.907,3.657,0.000,0.0000
6,7-8-1-2,1971.287,0.000,65.171,1971.287,0.000,65.084,1971.287,3.143,0.000,0.0000
7,7-8-1-3,2080.177,0.000,9.720,2080.177,0.000,9.723,2080.178,3.770,0.000,0.0000
8,7-9-1-1,2170.395,0.000,26.712,2170.395,0.000,26.477,2170.395,7.037,0.000,0.0000
9,7-9-1-2,2206.081,0.000,38.585,2206.081,0.000,39.063,2206.081,4.779,0.000,0.0000


In [196]:
exact_alns_agg_df

,instance_group,obj,gap,time,obj,gap,time,obj,gap,time,solver gap
0,5,2217.587,0.000,5.931,2217.587,0.000,5.895,2217.587,0.000,1.550,0.0000
1,7,2094.969,0.000,32.892,2094.969,0.000,32.870,2094.970,0.000,4.477,0.0000
2,9,5627.549,0.000,48.502,5627.549,0.000,48.607,5627.549,0.000,8.268,0.0000
3,11,3517.537,0.026,2417.191,5369.449,0.272,554.443,3517.537,0.000,50.258,0.0000
4,13,3994.678,0.079,3567.688,10025.767,0.654,600.324,3970.515,0.000,84.028,-0.0060
5,15,12477.863,0.258,3600.441,208617.891,20.237,600.041,8375.449,0.000,91.100,-0.3288
6,17,8567.211,0.610,3601.990,404884.820,40.412,600.049,4966.142,0.000,167.719,-0.4203
7,19,8988.366,0.611,3601.088,405441.420,40.494,600.151,5153.527,0.000,224.294,-0.4266
8,21,13325.404,0.445,3600.658,801358.178,80.074,600.108,9265.805,0.001,195.248,-0.3047
9,23,801846.912,80.157,1200.031,1000000.000,100.000,600.000,5962.097,0.000,363.911,-0.9926


## Local search operators

In [197]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    lso_df = generate_lso_df(run_ls_name)
    lso_file_name = f'dataframes/performance/lso.pkl'
    lso_df.to_pickle(lso_file_name)

In [198]:
lso_df = load_df('lso.pkl')
lso_df

,instance,one exchange,one relocate,two exchange,two relocate,postpone scheduled,schedule postponed,voyage exchange
0,5-5-1-1,464.40,372.60,0.00,0.000,0.0,0.000,0.000
1,5-5-1-2,707.00,0.00,0.00,0.000,0.0,0.000,0.000
2,5-6-1-1,0.00,0.00,0.00,0.000,0.0,0.000,0.000
3,5-7-1-1,1287.60,1871.20,0.00,0.000,0.0,0.000,0.000
4,5-7-1-2,478.20,716.80,0.00,0.000,0.0,0.000,0.000
5,7-8-1-1,106.40,72.40,0.00,0.000,0.0,0.000,0.000
6,7-8-1-2,6.00,10.40,0.00,0.000,0.0,0.000,0.000
7,7-8-1-3,48.80,398.80,0.00,0.000,0.0,0.000,0.000
8,7-9-1-1,430.40,1142.80,0.00,0.000,0.0,0.000,0.000
9,7-9-1-2,9.80,198.80,0.00,0.000,0.0,0.000,0.000
